In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer from the local directory
model_name = "./gpt2_local"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Test the model with some input
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Set pad_token_id to avoid warnings
pad_token_id = tokenizer.eos_token_id

# Create the attention mask
attention_mask = input_ids.ne(pad_token_id).long()

# Use top-k and top-p sampling
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    top_k=10,  # Top-k sampling
    top_p=0.9,  # Top-p (nucleus) sampling
    do_sample=True,  # Ensures sampling instead of greedy decoding
    pad_token_id=pad_token_id,  # Set the pad_token_id
    attention_mask=attention_mask  # Provide the attention mask
)

# Decode and print the output text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(generated_text)


Once upon a time, you could see it as a small, dim light in the darkness.

But now, as you were moving forward, you heard it as it approached, and as it passed, it stopped, and it began to rise


In [1]:
import numpy as np
import torch
import transformers

print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")

NumPy version: 1.26.4
PyTorch version: 2.0.1
Transformers version: 4.30.2


In [13]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", "gpt2-xl"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_prompt(seed_text=""):
    # Encode input prompt
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)

    # Generate text
    output = model.generate(
        input_ids,
        max_length=500,  # Adjust length of generated text
        num_return_sequences=1,
        no_repeat_ngram_size=5,
        top_k=40,
        top_p=0.5,
        temperature=1.9,
        do_sample=True,
    )

    # Decode and return the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage
if __name__ == "__main__":
    seed = " "
    prompt = generate_prompt(seed)
    print("One can only wish", prompt)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


God does not exist  2

(2) A man may take one or more of the following: (a) a small baggie of water; (b) a cup of water; (c) a small bag of tea; (d) a cup of milk; (e) an open jar of water; (f) a small piece of bread or small bag of flour; (g) a bowl of hot sauce; or (h) a bowl of cheese; or (i) a bottle or cup containing an amount equal to three grams of water per gallon of water. (4) A woman may use the word "sugar," "flour," or "oil," to mean a beverage that does not contain sugar or any other liquid or substance, but is prepared by boiling water. (5) An alcoholic beverage is a container that has been placed upon the container by the user or container is in an open container, if the person who made the beverage is a registered user or registered holder. (6) A person shall not have more than one person in any room, in any building or any place in which the person who makes the beverage is an employee of any of the following: a manufacturer or distributor of a food product for which t

In [25]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def get_word_probabilities(model, tokenizer, prompt):
    # Tokenize the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    # Get the model's output
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    
    # Get probabilities
    probs = torch.softmax(logits[0], dim=-1)
    
    # Get the tokens and their probabilities
    word_probs = []
    for i in range(input_ids.shape[1]):
        word = tokenizer.decode(input_ids[0][i])
        prob = probs[i][input_ids[0][i]].item()
        word_probs.append((word, prob))
    
    return word_probs

def main():
    # Load pre-trained model and tokenizer
    model_name = 'gpt2'
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    while True:
        prompt = input("Enter a prompt (or 'quit' to exit): ")
        if prompt.lower() == 'quit':
            break

        word_probs = get_word_probabilities(model, tokenizer, prompt)

        print("\nWord-by-word probability distribution:")
        for word, prob in word_probs:
            print(f"'{word}': {prob:.4f}")

if __name__ == "__main__":
    main()

Enter a prompt (or 'quit' to exit):  a cat is a



Word-by-word probability distribution:
'a': 0.0023
' cat': 0.0006
' is': 0.0001
' a': 0.0003


Enter a prompt (or 'quit' to exit):  a dog is the



Word-by-word probability distribution:
'a': 0.0023
' dog': 0.0014
' is': 0.0001
' the': 0.0003


Enter a prompt (or 'quit' to exit):  quit


In [38]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, top_k_top_p_filtering
import torch.nn.functional as F

# Load the model and tokenizer from the local directory
model_name = "./gpt2_local"  # Path to the locally stored GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Test the model with some input
input_text = "The quick brown fox"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Get the model's output logits
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Get the logits for the last token in the input
last_token_logits = logits[:, -1, :]
top_k = 10
top_p = 0.5
#####
temperature = 5
last_token_logits /= temperature
filtered_logits = top_k_top_p_filtering(last_token_logits, top_k=top_k, top_p=top_p)
# Calculate probabilities using softmax
probs = F.softmax(filtered_logits, dim=-1)

# Get the top 10 predictions
top_k = 10
top_k_probs, top_k_indices = torch.topk(probs, top_k)


print(f"Input: '{input_text}'")
print(f"Temperature: '{temperature}' ")
print(f"Top_p: '{top_p}' ")
print(f"Top_k: '{top_k}' ")
print(f"Top-{top_k} predictions for the next word:")

for i, (index, prob) in enumerate(zip(top_k_indices[0], top_k_probs[0])):
    predicted_word = tokenizer.decode([index])
    print(f"{i+1}: {predicted_word} (Probability: {prob.item():.4f})")


Input: 'The quick brown fox'
Temperature: '5' 
Top_p: '0.5' 
Top_k: '10' 
Top-10 predictions for the next word:
1: es (Probability: 0.2444)
2:  was (Probability: 0.1983)
3:  is (Probability: 0.1874)
4: 's (Probability: 0.1860)
5: , (Probability: 0.1839)
6: ' (Probability: 0.0000)
7: # (Probability: 0.0000)
8: & (Probability: 0.0000)
9: % (Probability: 0.0000)
10: * (Probability: 0.0000)
